# FraSoHome — Notebook 7: Baseline Modeling (Churn & Propensión)

> **Objetivo formativo:** construir **modelos baseline** rápidos y explicables (sin “mágia”), medir rendimiento con métricas estándar y discutir **fugas de información (data leakage)**, particionado temporal/grupal, umbrales, etc.

Este notebook asume que ya ejecutaste:
- **Notebook 5** (features) → genera `output_features/*.csv`
- **Notebook 6** (preprocesado) → genera `output_ml/*.csv`

Aun así, el notebook intenta ser **robusto** y te avisa si faltan archivos.

---

## Contenidos

1. Carga de datasets ML-ready (`output_ml/`)
2. Modelado baseline de **churn** (clientes)
3. Modelado baseline de **propensión de compra** (snapshots)
4. Comparación de modelos, umbral y explicabilidad simple
5. Export de métricas y (opcional) guardado del mejor modelo



In [ ]:
# ============================================
# 0) Setup
# ============================================
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report
)

try:
    from sklearn.inspection import permutation_importance
    _HAS_PERM = True
except Exception:
    _HAS_PERM = False

RANDOM_STATE = 42

BASE_DIR = Path(".")  # ajusta si ejecutas desde otra carpeta
OUTPUT_ML_DIR = BASE_DIR / "output_ml"
OUTPUT_MODELS_DIR = BASE_DIR / "output_models"
OUTPUT_MODELS_DIR.mkdir(parents=True, exist_ok=True)

CLIENTS_ML_PATH = OUTPUT_ML_DIR / "FraSoHome_clientes_ML_ready.csv"
PROP_ML_PATH = OUTPUT_ML_DIR / "FraSoHome_propension_ML_ready.csv"

print("📁 output_ml:", OUTPUT_ML_DIR.resolve())
print(" - clientes:", CLIENTS_ML_PATH.exists(), CLIENTS_ML_PATH)
print(" - propensión:", PROP_ML_PATH.exists(), PROP_ML_PATH)


## 1) Funciones reutilizables (estructura)

Todas las funciones están pensadas para ser **reutilizables** y trabajar con **dataframes como parámetros**.

- Carga robusta / estandarización ligera
- Detección del target
- Split (random / por grupo / temporal si existe `snapshot_date`)
- Entrenamiento y evaluación (AUC-ROC, PR-AUC, F1, etc.)
- Selección de umbral
- Importancia de variables (coeficientes, Gini, permutation importance)



In [ ]:
# ============================================
# 1) Helpers reutilizables
# ============================================

def standardize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def load_csv_robust(path: Path) -> pd.DataFrame:
    """Carga CSV como texto para no romperse con formatos raros; deja el tipado para después."""
    df = pd.read_csv(path, dtype=str, encoding="utf-8")
    df = standardize_column_names(df)
    return df

def infer_target_column(df: pd.DataFrame, preferred: str | None, candidates: list[str]) -> str | None:
    cols = set(df.columns)
    if preferred and preferred in cols:
        return preferred
    for c in candidates:
        if c in cols:
            return c
    # heurística: primer label_*
    label_cols = [c for c in df.columns if c.startswith("label_")]
    return label_cols[0] if label_cols else None

def to_numeric_df(X: pd.DataFrame) -> pd.DataFrame:
    """Intenta convertir todas las columnas a numérico; si no, coerciona a NaN."""
    Xn = X.copy()
    for c in Xn.columns:
        Xn[c] = pd.to_numeric(Xn[c], errors="coerce")
    return Xn

def split_X_y(
    df: pd.DataFrame,
    target_col: str,
    id_cols: list[str] | None = None,
    drop_cols: list[str] | None = None
) -> tuple[pd.DataFrame, pd.Series]:
    """Separa X e y; elimina IDs y columnas a descartar."""
    df = df.copy()
    y = df[target_col].copy()
    X = df.drop(columns=[target_col])

    cols_to_drop = set()
    if id_cols:
        cols_to_drop |= set([c for c in id_cols if c in X.columns])
    if drop_cols:
        cols_to_drop |= set([c for c in drop_cols if c in X.columns])

    if cols_to_drop:
        X = X.drop(columns=sorted(cols_to_drop))

    # Convierto a numérico (ML-ready debería venir ya numérico)
    X = to_numeric_df(X)

    # Target a 0/1
    y_num = pd.to_numeric(y, errors="coerce")
    if y_num.isna().any():
        # intenta mapear strings típicos
        y_map = (y.astype(str).str.strip().str.lower()
                 .replace({"true": 1, "false": 0, "yes": 1, "no": 0, "si": 1, "sí": 1, "n": 0, "s": 1}))
        y_num = pd.to_numeric(y_map, errors="coerce")
    return X, y_num

def print_basic_dataset_report(df: pd.DataFrame, name: str, target_col: str | None = None) -> None:
    print(f"\n=== {name} ===")
    print("shape:", df.shape)
    print("cols:", len(df.columns))
    print("nulos totales:", int(df.isna().sum().sum()))
    if target_col and target_col in df.columns:
        y = pd.to_numeric(df[target_col], errors="coerce")
        print("target:", target_col)
        print("clases (incl NaN):")
        print(y.value_counts(dropna=False).head(10))

def evaluate_binary_classifier(
    model,
    X_train: pd.DataFrame, y_train: pd.Series,
    X_test: pd.DataFrame, y_test: pd.Series,
    threshold: float = 0.5,
    name: str = "model"
) -> dict:
    """Entrena, predice y calcula métricas estándar."""
    model.fit(X_train, y_train)

    # proba positiva (col=1)
    if hasattr(model, "predict_proba"):
        p_test = model.predict_proba(X_test)[:, 1]
    else:
        # fallback a decision_function
        s = model.decision_function(X_test)
        p_test = (s - s.min()) / (s.max() - s.min() + 1e-9)

    y_pred = (p_test >= threshold).astype(int)

    metrics = {
        "model": name,
        "roc_auc": roc_auc_score(y_test, p_test) if y_test.nunique() > 1 else np.nan,
        "pr_auc": average_precision_score(y_test, p_test) if y_test.nunique() > 1 else np.nan,
        "accuracy": accuracy_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "threshold": threshold,
    }

    print(f"\n--- {name} @ threshold={threshold:.2f} ---")
    print(pd.Series(metrics).drop(["model"]).round(4))
    print("\nConfusion matrix (rows=true, cols=pred):")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification report:")
    print(classification_report(y_test, y_pred, zero_division=0))

    return {"metrics": metrics, "proba_test": p_test, "y_pred": y_pred, "fitted_model": model}

def threshold_sweep(y_true: pd.Series, y_prob: np.ndarray, metric: str = "f1") -> tuple[float, pd.DataFrame]:
    """Busca el mejor umbral para una métrica simple (f1 o recall o precision)."""
    y_true = pd.Series(y_true).astype(int).values
    rows = []
    for t in np.linspace(0.05, 0.95, 19):
        y_pred = (y_prob >= t).astype(int)
        row = {
            "threshold": float(t),
            "f1": f1_score(y_true, y_pred, zero_division=0),
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "accuracy": accuracy_score(y_true, y_pred),
        }
        rows.append(row)
    df = pd.DataFrame(rows)
    if metric not in df.columns:
        metric = "f1"
    best_t = float(df.sort_values(metric, ascending=False).iloc[0]["threshold"])
    return best_t, df.sort_values(metric, ascending=False)

def plot_score_distributions(y_true: pd.Series, y_prob: np.ndarray, title: str) -> None:
    y_true = pd.Series(y_true).astype(int).values
    plt.figure()
    plt.hist(y_prob[y_true == 0], bins=20, alpha=0.6, label="Clase 0")
    plt.hist(y_prob[y_true == 1], bins=20, alpha=0.6, label="Clase 1")
    plt.title(title)
    plt.xlabel("Probabilidad estimada")
    plt.ylabel("Frecuencia")
    plt.legend()
    plt.show()

def safe_feature_importance(model, feature_names: list[str], top_n: int = 20) -> pd.DataFrame:
    """Devuelve importancias si el modelo las tiene (coef o feature_importances_)."""
    rows = []
    m = model
    # si es pipeline, toma el último paso
    if hasattr(model, "named_steps"):
        m = list(model.named_steps.values())[-1]

    if hasattr(m, "coef_"):
        coefs = m.coef_.ravel()
        rows = [{"feature": f, "importance": float(w)} for f, w in zip(feature_names, coefs)]
    elif hasattr(m, "feature_importances_"):
        imps = m.feature_importances_
        rows = [{"feature": f, "importance": float(w)} for f, w in zip(feature_names, imps)]
    else:
        return pd.DataFrame(columns=["feature", "importance"])

    df = pd.DataFrame(rows)
    df["abs_importance"] = df["importance"].abs()
    return df.sort_values("abs_importance", ascending=False).head(top_n)[["feature", "importance", "abs_importance"]]

def permutation_importance_report(model, X_test: pd.DataFrame, y_test: pd.Series, top_n: int = 20) -> pd.DataFrame:
    if not _HAS_PERM:
        print("ℹ️ permutation_importance no disponible en este entorno.")
        return pd.DataFrame(columns=["feature", "importance_mean", "importance_std"])

    r = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1)
    df = pd.DataFrame({
        "feature": X_test.columns,
        "importance_mean": r.importances_mean,
        "importance_std": r.importances_std
    })
    return df.sort_values("importance_mean", ascending=False).head(top_n)


## 2) Caso de uso 1: **Churn** (clientes)

Trabajaremos con `output_ml/FraSoHome_clientes_ML_ready.csv`.

- **Target esperado:** `label_churn_180d` (0/1)
- Se eliminan columnas ID si existen (`customer_id`, etc.)
- Split estratificado train/test
- Comparación de baselines:
  - Dummy (mayoría)
  - Logistic Regression (baseline interpretable)
  - RandomForest (baseline no lineal)
  - GradientBoosting (baseline boosting clásico)

> Nota formativa: al ser un dataset sintético y con “ruido” intencional, las métricas pueden variar bastante. Lo importante es practicar el **proceso**.



In [ ]:
# ============================================
# 2) Carga dataset churn
# ============================================
if not CLIENTS_ML_PATH.exists():
    raise FileNotFoundError(
        f"No existe {CLIENTS_ML_PATH}. Ejecuta antes el Notebook 6 (output_ml/) o ajusta BASE_DIR."
    )

df_churn = load_csv_robust(CLIENTS_ML_PATH)
print_basic_dataset_report(df_churn, "clientes_ml_ready", target_col="label_churn_180d")

# Detecta target
target_churn = infer_target_column(
    df_churn,
    preferred="label_churn_180d",
    candidates=["target", "y", "churn", "will_churn"]
)
if not target_churn:
    raise ValueError("No se encontró columna target para churn. Revisa el dataset.")

# IDs típicos: si existen, los quitamos de X
id_cols_churn = [c for c in df_churn.columns if c in {"customer_id", "cliente_id", "id_cliente"} or c.endswith("_id")]

# Construye X,y
X_churn, y_churn = split_X_y(df_churn, target_col=target_churn, id_cols=id_cols_churn)

# Elimina filas donde y sea NaN (si el label venía corrupto)
mask = y_churn.notna()
X_churn = X_churn.loc[mask].copy()
y_churn = y_churn.loc[mask].astype(int).copy()

print("\nX_churn shape:", X_churn.shape)
print("y_churn balance:")
print(y_churn.value_counts(normalize=True).round(3))

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X_churn, y_churn,
    test_size=0.25,
    random_state=RANDOM_STATE,
    stratify=y_churn
)

print("\nTrain:", X_train.shape, "Test:", X_test.shape)


In [ ]:
# ============================================
# 2.1) Modelos baseline churn
# ============================================
models_churn = {
    "dummy_most_frequent": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE))
    ]),
    "logreg_balanced": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        # Si Notebook 6 ya escaló, esto no es imprescindible. Se deja por robustez.
        ("scaler", StandardScaler(with_mean=False)),
        ("model", LogisticRegression(max_iter=2000, class_weight="balanced", random_state=RANDOM_STATE))
    ]),
    "random_forest": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", RandomForestClassifier(
            n_estimators=400,
            random_state=RANDOM_STATE,
            n_jobs=-1,
            class_weight="balanced_subsample"
        ))
    ]),
    "grad_boost": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", GradientBoostingClassifier(random_state=RANDOM_STATE))
    ]),
}

results_churn = []
fitted_churn = {}

for name, model in models_churn.items():
    out = evaluate_binary_classifier(
        model=model,
        X_train=X_train, y_train=y_train,
        X_test=X_test, y_test=y_test,
        threshold=0.50,
        name=name
    )
    results_churn.append(out["metrics"])
    fitted_churn[name] = out

df_results_churn = pd.DataFrame(results_churn).sort_values(["roc_auc", "pr_auc"], ascending=False)
df_results_churn


In [ ]:
# ============================================
# 2.2) Ajuste simple de umbral (sobre el mejor modelo por ROC-AUC)
# ============================================
best_model_name = df_results_churn.iloc[0]["model"]
best = fitted_churn[best_model_name]
p_test = best["proba_test"]

best_t, sweep = threshold_sweep(y_test, p_test, metric="f1")
print("Mejor umbral (F1):", best_t)
sweep.head(10)

# Re-eval con el umbral óptimo
_ = evaluate_binary_classifier(
    model=best["fitted_model"],
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test,
    threshold=best_t,
    name=f"{best_model_name}_tuned"
)

plot_score_distributions(y_test, p_test, title=f"Distribución de scores - {best_model_name}")


In [ ]:
# ============================================
# 2.3) Explicabilidad simple (importancia de variables)
# ============================================
# OJO: en pipelines con scaler, las importancias siguen siendo interpretables (coef), pero en escala estandarizada.
best_fitted = best["fitted_model"]
top_imp = safe_feature_importance(best_fitted, feature_names=list(X_train.columns), top_n=25)
print("Top importancias (si aplica):")
display(top_imp)

if _HAS_PERM:
    perm = permutation_importance_report(best_fitted, X_test, y_test, top_n=25)
    print("\nPermutation importance (top 25):")
    display(perm)


In [ ]:
# ============================================
# 2.4) Export de métricas churn + (opcional) guardado del modelo
# ============================================
churn_metrics_path = OUTPUT_MODELS_DIR / "churn_metrics_baseline.csv"
df_results_churn.to_csv(churn_metrics_path, index=False, encoding="utf-8")
print("✅ Exportado:", churn_metrics_path)

# Guardar modelo (joblib)
try:
    import joblib
    churn_model_path = OUTPUT_MODELS_DIR / f"churn_model_{best_model_name}.joblib"
    joblib.dump(best_fitted, churn_model_path)
    print("✅ Modelo guardado:", churn_model_path)
except Exception as e:
    print("ℹ️ No se pudo guardar el modelo (joblib). Motivo:", e)


## 3) Caso de uso 2: **Propensión de compra** (snapshots)

Trabajaremos con `output_ml/FraSoHome_propension_ML_ready.csv` si existe.

- El notebook intenta detectar el **target** automáticamente:
  - primera columna `label_*`, o `target`, `y`, `will_buy`
- Split **recomendado** (formativo):
  - si existe `snapshot_date`: split **temporal** (último tramo como test)
  - si existe `customer_id`: split por **grupo** (evitar leakage por cliente)
  - si no: split aleatorio estratificado

> Esto es crucial: en problemas de propensión, un split aleatorio puede “filtrar futuro” de forma implícita si hay múltiples snapshots del mismo cliente.



In [ ]:
# ============================================
# 3) Carga dataset propensión (si existe)
# ============================================
if not PROP_ML_PATH.exists():
    print("ℹ️ No existe dataset de propensión ML-ready. Ejecuta Notebook 5+6 para generarlo.")
    df_prop = None
else:
    df_prop = load_csv_robust(PROP_ML_PATH)
    print_basic_dataset_report(df_prop, "propension_ml_ready", target_col=None)

df_prop


In [ ]:
# ============================================
# 3.1) Split recomendado (temporal/grupal si se puede)
# ============================================
def split_for_propensity(
    df: pd.DataFrame,
    target_col: str,
    date_col: str | None = "snapshot_date",
    group_col: str | None = "customer_id",
    test_size: float = 0.25
):
    df = df.copy()

    # si hay fecha, intenta split temporal
    if date_col and date_col in df.columns:
        dt = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)
        df["_snapshot_dt"] = dt
        df = df.sort_values("_snapshot_dt")

        # cutoff por percentil (último 25% como test)
        cutoff = df["_snapshot_dt"].quantile(1 - test_size)
        train_df = df[df["_snapshot_dt"] <= cutoff].copy()
        test_df = df[df["_snapshot_dt"] > cutoff].copy()

        # si por errores queda vacío, fallback
        if len(train_df) > 50 and len(test_df) > 10:
            return train_df, test_df, f"temporal (cutoff={cutoff.date() if pd.notna(cutoff) else cutoff})"

    # si hay grupo, split por cliente
    if group_col and group_col in df.columns:
        gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=RANDOM_STATE)
        groups = df[group_col].astype(str)
        idx_train, idx_test = next(gss.split(df, groups=groups))
        train_df = df.iloc[idx_train].copy()
        test_df = df.iloc[idx_test].copy()
        return train_df, test_df, f"group ({group_col})"

    # fallback aleatorio estratificado
    train_df, test_df = train_test_split(
        df, test_size=test_size, random_state=RANDOM_STATE,
        stratify=pd.to_numeric(df[target_col], errors="coerce")
    )
    return train_df.copy(), test_df.copy(), "random stratified"

if df_prop is None:
    print("ℹ️ Saltando sección de propensión.")
else:
    # target
    target_prop = infer_target_column(df_prop, preferred=None, candidates=["target", "y", "will_buy"])
    if not target_prop:
        raise ValueError("No se pudo inferir target en propensión. Añade una columna label_* en el dataset.")

    # IDs potenciales
    id_cols_prop = [c for c in df_prop.columns if c.endswith("_id") or c in {"customer_id", "product_id", "snapshot_date"}]

    # Pre-split (para preservar snapshot_date/customer_id antes de convertir a numérico)
    train_df, test_df, split_strategy = split_for_propensity(df_prop, target_col=target_prop)
    print("✅ Split estrategia:", split_strategy)
    print("train/test:", train_df.shape, test_df.shape)

    # X,y para train y test
    X_train_p, y_train_p = split_X_y(train_df, target_col=target_prop, id_cols=id_cols_prop)
    X_test_p, y_test_p = split_X_y(test_df, target_col=target_prop, id_cols=id_cols_prop)

    # limpia NaNs en y
    mtr = y_train_p.notna()
    mte = y_test_p.notna()
    X_train_p, y_train_p = X_train_p.loc[mtr], y_train_p.loc[mtr].astype(int)
    X_test_p, y_test_p = X_test_p.loc[mte], y_test_p.loc[mte].astype(int)

    print("X_train_p:", X_train_p.shape, "X_test_p:", X_test_p.shape)
    print("Balance train:", y_train_p.value_counts(normalize=True).round(3).to_dict())


In [ ]:
# ============================================
# 3.2) Modelos baseline propensión
# ============================================
if df_prop is None:
    pass
else:
    models_prop = {
        "dummy_most_frequent": Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("model", DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE))
        ]),
        "logreg_balanced": Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler(with_mean=False)),
            ("model", LogisticRegression(max_iter=2000, class_weight="balanced", random_state=RANDOM_STATE))
        ]),
        "random_forest": Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("model", RandomForestClassifier(
                n_estimators=400,
                random_state=RANDOM_STATE,
                n_jobs=-1,
                class_weight="balanced_subsample"
            ))
        ]),
        "grad_boost": Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("model", GradientBoostingClassifier(random_state=RANDOM_STATE))
        ]),
    }

    results_prop = []
    fitted_prop = {}

    for name, model in models_prop.items():
        out = evaluate_binary_classifier(
            model=model,
            X_train=X_train_p, y_train=y_train_p,
            X_test=X_test_p, y_test=y_test_p,
            threshold=0.50,
            name=name
        )
        results_prop.append(out["metrics"])
        fitted_prop[name] = out

    df_results_prop = pd.DataFrame(results_prop).sort_values(["roc_auc", "pr_auc"], ascending=False)
    df_results_prop


In [ ]:
# ============================================
# 3.3) Umbral + importancia (mejor modelo)
# ============================================
if df_prop is None:
    pass
else:
    best_prop_model_name = df_results_prop.iloc[0]["model"]
    best_prop = fitted_prop[best_prop_model_name]
    p_test_prop = best_prop["proba_test"]

    best_t_prop, sweep_prop = threshold_sweep(y_test_p, p_test_prop, metric="f1")
    print("Mejor umbral (F1) propensión:", best_t_prop)
    display(sweep_prop.head(10))

    _ = evaluate_binary_classifier(
        model=best_prop["fitted_model"],
        X_train=X_train_p, y_train=y_train_p,
        X_test=X_test_p, y_test=y_test_p,
        threshold=best_t_prop,
        name=f"{best_prop_model_name}_tuned"
    )

    plot_score_distributions(y_test_p, p_test_prop, title=f"Distribución de scores - {best_prop_model_name}")

    # Importancias
    top_imp_prop = safe_feature_importance(best_prop["fitted_model"], feature_names=list(X_train_p.columns), top_n=25)
    print("Top importancias (si aplica):")
    display(top_imp_prop)

    if _HAS_PERM:
        perm_prop = permutation_importance_report(best_prop["fitted_model"], X_test_p, y_test_p, top_n=25)
        print("\nPermutation importance (top 25):")
        display(perm_prop)


In [ ]:
# ============================================
# 3.4) Export métricas propensión + guardado modelo
# ============================================
if df_prop is None:
    pass
else:
    prop_metrics_path = OUTPUT_MODELS_DIR / "propension_metrics_baseline.csv"
    df_results_prop.to_csv(prop_metrics_path, index=False, encoding="utf-8")
    print("✅ Exportado:", prop_metrics_path)

    try:
        import joblib
        prop_model_path = OUTPUT_MODELS_DIR / f"propension_model_{best_prop_model_name}.joblib"
        joblib.dump(best_prop["fitted_model"], prop_model_path)
        print("✅ Modelo guardado:", prop_model_path)
    except Exception as e:
        print("ℹ️ No se pudo guardar el modelo (joblib). Motivo:", e)


## 4) Siguientes pasos (para el curso)

**Ideas de ampliación para prácticas:**
- Validación más realista:
  - churn: split temporal (train en meses antiguos, test en meses recientes)
  - propensión: split por cliente + temporal
- Tratamiento de desbalance:
  - `class_weight`, *undersampling/oversampling*, métricas PR-AUC
- Ingeniería de variables adicional:
  - features por categoría preferida, tasa de descuento, estacionalidad
- Modelos adicionales:
  - XGBoost/LightGBM (si se permite en el curso)
- Trazabilidad:
  - guardar el `threshold` elegido y un diccionario de columnas para scoring en producción

> Importante: en un caso real, los features deben calcularse “mirando hacia atrás” (ventanas históricas) para evitar **leakage**.

